# 资金费率与基差系统研究

**从基础到高阶的完整学习指南**

---

## 目录

### 第一部分：基础概念
1. 永续合约简介
2. 资金费率（Funding Rate）基础
3. 基差（Basis）基础

### 第二部分：深入理解
4. 资金费率计算公式与机制
5. 基差的计算与意义
6. 资金费率与基差的理论关系

### 第三部分：实证分析
7. 数据获取与处理
8. 资金费率与基差的相关性分析
9. 时间序列特征分析

### 第四部分：高阶应用
10. 套利策略设计
11. 预测模型构建
12. 风险管理与实战建议

---

# 第一部分：基础概念

---

## 1. 永续合约简介

### 1.1 什么是永续合约？

**永续合约（Perpetual Swap/Perpetual Futures）** 是一种没有到期日的衍生品合约，允许交易者无限期持有头寸。

### 1.2 永续合约 vs 传统期货

| 特性 | 传统期货 | 永续合约 |
|------|----------|----------|
| 到期日 | 有固定到期日 | **无到期日** |
| 交割方式 | 到期交割（实物/现金） | **无需交割** |
| 价格锚定 | 到期时收敛于现货 | **通过资金费率锚定** |
| 持仓成本 | 展期成本 | **资金费率** |
| 杠杆 | 通常较低 | 可达 125x |

### 1.3 为什么需要资金费率？

传统期货通过到期交割机制使价格最终收敛于现货价格。但永续合约没有到期日，如果没有其他机制，合约价格可能会与现货价格产生显著偏离。

**资金费率的作用：**
- 当合约价格 > 现货价格时，多头支付空头 → 抑制做多
- 当合约价格 < 现货价格时，空头支付多头 → 抑制做空
- 通过这种激励机制使永续合约价格锚定现货价格

## 2. 资金费率（Funding Rate）基础

### 2.1 定义

**资金费率** 是永续合约多空双方之间定期交换的费用，用于使合约价格与现货价格保持一致。

### 2.2 核心要点

```
┌─────────────────────────────────────────────────────────────┐
│                      资金费率机制                            │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   正资金费率 (Funding Rate > 0)                             │
│   ├── 含义：合约价格 > 现货价格（溢价）                      │
│   ├── 支付方向：多头 → 空头                                 │
│   └── 市场情绪：看涨情绪过热                                │
│                                                             │
│   负资金费率 (Funding Rate < 0)                             │
│   ├── 含义：合约价格 < 现货价格（折价）                      │
│   ├── 支付方向：空头 → 多头                                 │
│   └── 市场情绪：看跌情绪过热                                │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 2.3 结算周期

- **Binance**: 每 8 小时结算一次（00:00, 08:00, 16:00 UTC）
- **OKX**: 每 8 小时
- **Bybit**: 每 8 小时
- **dYdX**: 每 1 小时

### 2.4 费用计算

```
资金费用 = 持仓价值 × 资金费率
        = 合约数量 × 标记价格 × 资金费率
```

**示例：**
- 持有 1 BTC 多头，标记价格 $50,000
- 资金费率 = 0.01%
- 资金费用 = 1 × 50,000 × 0.0001 = **$5**（多头支付）

## 3. 基差（Basis）基础

### 3.1 定义

**基差（Basis）** = 期货/合约价格 - 现货价格

### 3.2 基差的类型

| 类型 | 条件 | 含义 |
|------|------|------|
| **正基差（Contango）** | 合约价格 > 现货价格 | 市场预期上涨，多头情绪 |
| **负基差（Backwardation）** | 合约价格 < 现货价格 | 市场预期下跌，空头情绪 |
| **零基差** | 合约价格 ≈ 现货价格 | 市场平衡 |

### 3.3 基差率

为了便于不同价格资产的比较，通常使用**基差率（Basis Rate）**：

```
基差率 = (合约价格 - 现货价格) / 现货价格 × 100%
```

### 3.4 基差与年化收益

```
年化基差率 = 基差率 × (365 / 持有天数) × 100%

对于永续合约（8小时结算）：
年化基差率 ≈ 基差率 × 3 × 365 × 100%
```

---

# 第二部分：深入理解

---

## 4. 资金费率计算公式与机制

### 4.1 Binance 资金费率计算公式

Binance 永续合约的资金费率由两部分组成：

```
资金费率 = clamp(溢价指数 + clamp(利率 - 溢价指数, -0.05%, 0.05%), -0.75%, 0.75%)
```

简化版：
```
资金费率 ≈ 溢价指数 + 利率调整
```

### 4.2 组成部分详解

#### (1) 利率（Interest Rate）
- 代表持有标的资产与计价货币的利率差
- Binance 默认：**0.03% / 天 = 0.01% / 8小时**
- 这是一个固定值，反映借贷成本

#### (2) 溢价指数（Premium Index）
- 反映永续合约与现货价格的偏离程度
- 计算公式：

```
溢价指数 = [Max(0, 买一价 - 公允价格) - Max(0, 公允价格 - 卖一价)] / 现货价格

公允价格 = 现货价格 × (1 + 利率 × 剩余结算时间/8小时)
```

### 4.3 资金费率的上下限

| 交易所 | 最大正费率 | 最大负费率 |
|--------|-----------|------------|
| Binance | +0.75% / 8h | -0.75% / 8h |
| OKX | +1.5% / 8h | -1.5% / 8h |
| Bybit | +0.75% / 8h | -0.75% / 8h |

### 4.4 资金费率的影响因素

```
┌────────────────────────────────────────────────────────────────┐
│                     影响资金费率的因素                          │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  1. 市场情绪                                                   │
│     └── 牛市：正费率居多（多头支付）                           │
│     └── 熊市：负费率增加（空头支付）                           │
│                                                                │
│  2. 供需关系                                                   │
│     └── 多头需求 > 空头供给 → 正费率                           │
│     └── 空头需求 > 多头供给 → 负费率                           │
│                                                                │
│  3. 杠杆使用率                                                 │
│     └── 高杠杆多头增加 → 费率上升                              │
│                                                                │
│  4. 跨交易所套利                                               │
│     └── 套利者会平抑过高的费率                                 │
│                                                                │
│  5. 宏观事件                                                   │
│     └── 重大消息可能导致费率剧烈波动                           │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

## 5. 基差的计算与意义

### 5.1 基差计算方法

#### 绝对基差
```python
绝对基差 = 永续合约价格 - 现货价格
```

#### 基差率（更常用）
```python
基差率 = (永续合约价格 - 现货价格) / 现货价格 × 100%
```

#### 年化基差率
```python
# 对于8小时结算的永续合约
年化基差率 = 基差率 × 3 × 365
```

### 5.2 基差的市场含义

| 基差状态 | 市场解读 | 交易信号 |
|----------|----------|----------|
| 高正基差 | 市场极度看涨 | 可能过热，谨慎追多 |
| 低正基差 | 温和看涨 | 正常市场状态 |
| 零基差 | 市场平衡 | 无明显方向 |
| 低负基差 | 温和看跌 | 正常市场状态 |
| 高负基差 | 市场极度恐慌 | 可能超卖，关注反弹 |

### 5.3 基差与持仓成本

对于永续合约持有者：
- **做多者**：正基差意味着需要支付溢价
- **做空者**：正基差意味着可以获得贴水收益

### 5.4 基差的收敛机制

```
┌─────────────────────────────────────────────────────────────────┐
│                      基差收敛过程                               │
├─────────────────────────────────────��───────────────────────────┤
│                                                                 │
│   正基差过大                                                    │
│      ↓                                                          │
│   资金费率上升（多头支付更多）                                  │
│      ↓                                                          │
│   部分多头平仓 + 套利者入场做空                                 │
│      ↓                                                          │
│   合约价格下降，向现货价格靠拢                                  │
│      ↓                                                          │
│   基差收窄                                                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## 6. 资金费率与基差的理论关系

### 6.1 核心关系

**资金费率 ≈ f(基差) + 利率**

资金费率与基差高度相关，但并非完全等同：

```
┌────────────────────────────────────────────────────────────────┐
│                                                                │
│    基差（Basis）                 资金费率（Funding Rate）      │
│         │                               │                      │
│         │  ┌─────────────────────────┐  │                      │
│         └──│  正相关但不完全相等      │──┘                      │
│            └─────────────────────────┘                         │
│                                                                │
│    原因：                                                      │
│    1. 资金费率有上下限（±0.75%）                               │
│    2. 资金费率包含固定利率成分                                 │
│    3. 计算使用时间加权平均                                     │
│    4. 存在滞后效应                                             │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

### 6.2 数学关系

在理想情况下：
```
资金费率 ≈ 基差率 / 结算间隔时间 + 利率

对于 Binance（8小时结算）：
资金费率 ≈ 基差率 / 8 + 0.01%
```

### 6.3 相互影响机制

#### 基差 → 资金费率
- 正基差（溢价）→ 正资金费率
- 负基差（折价）→ 负资金费率
- 基差越大，资金费率的绝对值越大（但有上限）

#### 资金费率 → 基差
- 高资金费率 → 抑制多头 → 合约价格下降 → 基差收窄
- 低/负资金费率 → 抑制空头 → 合约价格上升 → 基差收窄

### 6.4 不一致的情况

有时资金费率和基差可能出现"不一致"的情况：

| 情况 | 可能��因 |
|------|----------|
| 高基差 + 低费率 | 基差刚形成，费率还未调整 |
| 低基差 + 高费率 | 费率刚结算，正在收敛 |
| 正基差 + 负费率 | 极端市场波动，利率占主导 |

---

# 第三部分：实证分析

---

## 7. 数据获取与处理

In [ ]:
# 7.1 导入依赖库

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime, timedelta
import warnings
from scipy import stats

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print('✅ 所有依赖库导入成功！')

In [ ]:
# 7.2 初始化交易所连接

# 现货交易所
exchange_spot = ccxt.binance({
    'enableRateLimit': True,
    'options': {'defaultType': 'spot'}
})

# 永续合约交易所
exchange_futures = ccxt.binance({
    'enableRateLimit': True,
    'options': {'defaultType': 'swap'}
})

# 加载市场
exchange_spot.load_markets()
exchange_futures.load_markets()

print('✅ 交易所连接成功！')
print(f'   现货市场: {len(exchange_spot.markets)} 个')
print(f'   合约市场: {len(exchange_futures.markets)} 个')

In [ ]:
# 7.3 定义数据获取函数

def get_funding_rate_history(symbol: str, days: int = 90) -> pd.DataFrame:
    """
    获取历史资金费率
    
    参数:
        symbol: 交易对，如 'BTC/USDT:USDT'
        days: 获取天数
    
    返回:
        DataFrame: 包含时间戳、资金费率等信息
    """
    since = int((datetime.now() - timedelta(days=days)).timestamp() * 1000)
    
    all_rates = []
    while True:
        rates = exchange_futures.fetch_funding_rate_history(symbol, since=since, limit=1000)
        if not rates:
            break
        all_rates.extend(rates)
        since = rates[-1]['timestamp'] + 1
        if len(rates) < 1000:
            break
    
    df = pd.DataFrame(all_rates)
    if not df.empty:
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.set_index('datetime')
        # 添加衍生列
        df['funding_rate_pct'] = df['fundingRate'] * 100  # 百分比
        df['funding_rate_annualized'] = df['fundingRate'] * 3 * 365 * 100  # 年化百分比
    
    return df


def get_current_basis(symbol: str) -> dict:
    """
    获取当前基差信息
    
    返回:
        dict: 包含现货价格、合约价格、基差、基差率等
    """
    # 获取价格
    spot_symbol = symbol.replace(':USDT', '')
    spot_ticker = exchange_spot.fetch_ticker(spot_symbol)
    futures_ticker = exchange_futures.fetch_ticker(symbol)
    
    spot_price = spot_ticker['last']
    futures_price = futures_ticker['last']
    
    # 计算基差
    basis = futures_price - spot_price
    basis_rate = (basis / spot_price) * 100
    basis_annualized = basis_rate * 3 * 365  # 假设8小时周期
    
    return {
        'symbol': symbol,
        'spot_price': spot_price,
        'futures_price': futures_price,
        'basis': basis,
        'basis_rate': basis_rate,
        'basis_annualized': basis_annualized
    }


def get_current_funding_rate(symbol: str) -> dict:
    """
    获取当前资金费率
    """
    funding = exchange_futures.fetch_funding_rate(symbol)
    
    return {
        'symbol': symbol,
        'funding_rate': funding['fundingRate'] * 100,
        'funding_rate_annualized': funding['fundingRate'] * 3 * 365 * 100,
        'next_funding_time': datetime.fromtimestamp(funding['fundingTimestamp'] / 1000)
    }


print('✅ 数据获取函数定义完成！')

In [ ]:
# 7.4 获取实时数据快照

# 研究的交易对
SYMBOLS = [
    'BTC/USDT:USDT',
    'ETH/USDT:USDT',
    'SOL/USDT:USDT',
    'BNB/USDT:USDT',
    'XRP/USDT:USDT',
]

print('📊 获取实时资金费率和基差数据...\n')
print(f'{"交易对":<15} {"资金费率":<12} {"年化费率":<12} {"基差率":<12} {"年化基差":<12}')
print('-' * 65)

current_data = []
for symbol in SYMBOLS:
    try:
        funding = get_current_funding_rate(symbol)
        basis = get_current_basis(symbol)
        
        row = {
            'symbol': symbol.replace('/USDT:USDT', ''),
            'funding_rate': funding['funding_rate'],
            'funding_annualized': funding['funding_rate_annualized'],
            'basis_rate': basis['basis_rate'],
            'basis_annualized': basis['basis_annualized'],
            'spot_price': basis['spot_price'],
            'futures_price': basis['futures_price'],
        }
        current_data.append(row)
        
        print(f"{row['symbol']:<15} {row['funding_rate']:>8.4f}%   {row['funding_annualized']:>8.2f}%   "
              f"{row['basis_rate']:>8.4f}%   {row['basis_annualized']:>8.2f}%")
    except Exception as e:
        print(f'{symbol}: 获取失败 - {e}')

df_current = pd.DataFrame(current_data)
print('\n✅ 实时数据获取完成！')

In [ ]:
# 7.5 获取 BTC 历史资金费率（用于后续分析）

print('📊 获取 BTC 历史资金费率数据...')
df_btc_funding = get_funding_rate_history('BTC/USDT:USDT', days=180)
print(f'✅ 获取到 {len(df_btc_funding)} 条记录')
print(f'   时间范围: {df_btc_funding.index[0]} ~ {df_btc_funding.index[-1]}')

# 显示基本统计
print('\n📈 基本统计:')
print(f'   平均费率: {df_btc_funding["funding_rate_pct"].mean():.4f}%')
print(f'   最大费率: {df_btc_funding["funding_rate_pct"].max():.4f}%')
print(f'   最小费率: {df_btc_funding["funding_rate_pct"].min():.4f}%')
print(f'   正费率比例: {(df_btc_funding["fundingRate"] > 0).mean()*100:.1f}%')

## 8. 资金费率与基差的相关性分析

In [ ]:
# 8.1 可视化当前资金费率与基差的关系

if not df_current.empty:
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('资金费率 vs 基差率', '年化费率 vs 年化基差'),
        horizontal_spacing=0.15
    )
    
    # 子图1: 原始费率
    fig.add_trace(
        go.Scatter(
            x=df_current['basis_rate'],
            y=df_current['funding_rate'],
            mode='markers+text',
            text=df_current['symbol'],
            textposition='top center',
            marker=dict(size=15, color='blue'),
            name='当前数据'
        ),
        row=1, col=1
    )
    
    # 子图2: 年化费率
    fig.add_trace(
        go.Scatter(
            x=df_current['basis_annualized'],
            y=df_current['funding_annualized'],
            mode='markers+text',
            text=df_current['symbol'],
            textposition='top center',
            marker=dict(size=15, color='green'),
            name='年化数据'
        ),
        row=1, col=2
    )
    
    # 添加参考线
    fig.add_hline(y=0, line_dash='dash', line_color='gray', row=1, col=1)
    fig.add_vline(x=0, line_dash='dash', line_color='gray', row=1, col=1)
    fig.add_hline(y=0, line_dash='dash', line_color='gray', row=1, col=2)
    fig.add_vline(x=0, line_dash='dash', line_color='gray', row=1, col=2)
    
    fig.update_xaxes(title_text='基差率 (%)', row=1, col=1)
    fig.update_yaxes(title_text='资金费率 (%)', row=1, col=1)
    fig.update_xaxes(title_text='年化基差率 (%)', row=1, col=2)
    fig.update_yaxes(title_text='年化资金费率 (%)', row=1, col=2)
    
    fig.update_layout(
        title='资金费率与基差的关系（当前快照）',
        height=500,
        showlegend=False
    )
    
    fig.show()
    
    # 计算相关系数
    if len(df_current) > 2:
        corr = df_current['funding_rate'].corr(df_current['basis_rate'])
        print(f'\n📊 资金费率与基差率的相关系数: {corr:.4f}')

In [ ]:
# 8.2 获取历史基差数据（需要通过价格计算）

def get_historical_prices(symbol: str, days: int = 90) -> pd.DataFrame:
    """
    获取历史价格数据（现货和合约）
    """
    since = int((datetime.now() - timedelta(days=days)).timestamp() * 1000)
    
    # 获取现货价格
    spot_symbol = symbol.replace(':USDT', '')
    spot_ohlcv = exchange_spot.fetch_ohlcv(spot_symbol, '1h', since=since, limit=1000)
    
    # 获取合约价格
    futures_ohlcv = exchange_futures.fetch_ohlcv(symbol, '1h', since=since, limit=1000)
    
    # 转换为 DataFrame
    df_spot = pd.DataFrame(spot_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df_futures = pd.DataFrame(futures_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    
    df_spot['datetime'] = pd.to_datetime(df_spot['timestamp'], unit='ms')
    df_futures['datetime'] = pd.to_datetime(df_futures['timestamp'], unit='ms')
    
    # 合并
    df = pd.merge(
        df_spot[['datetime', 'close']].rename(columns={'close': 'spot_price'}),
        df_futures[['datetime', 'close']].rename(columns={'close': 'futures_price'}),
        on='datetime',
        how='inner'
    )
    
    df = df.set_index('datetime')
    
    # 计算基差
    df['basis'] = df['futures_price'] - df['spot_price']
    df['basis_rate'] = (df['basis'] / df['spot_price']) * 100
    
    return df

print('📊 获取 BTC 历史价格数据...')
df_btc_prices = get_historical_prices('BTC/USDT:USDT', days=90)
print(f'✅ 获取到 {len(df_btc_prices)} 条记录')

In [ ]:
# 8.3 合并资金费率和基差数据

# 将资金费率数据与价格数据对齐
# 资金费率每8小时结算，我们需要将其扩展到小时级别

df_funding_hourly = df_btc_funding[['fundingRate', 'funding_rate_pct']].copy()
df_funding_hourly = df_funding_hourly.resample('1h').ffill()  # 前向填充

# 合并
df_analysis = pd.merge(
    df_btc_prices,
    df_funding_hourly,
    left_index=True,
    right_index=True,
    how='inner'
)

print(f'✅ 合并后数据: {len(df_analysis)} 条记录')
print(f'   时间范围: {df_analysis.index[0]} ~ {df_analysis.index[-1]}')

df_analysis.head()

In [ ]:
# 8.4 资金费率与基差的相关性分析

# 计算相关系数
corr_pearson = df_analysis['funding_rate_pct'].corr(df_analysis['basis_rate'])
corr_spearman = df_analysis['funding_rate_pct'].corr(df_analysis['basis_rate'], method='spearman')

print('=' * 60)
print('📊 资金费率与基差的相关性分析')
print('=' * 60)
print(f'\nPearson 相关系数:  {corr_pearson:.4f}')
print(f'Spearman 相关系数: {corr_spearman:.4f}')

# 解读
if abs(corr_pearson) > 0.7:
    strength = '强'
elif abs(corr_pearson) > 0.4:
    strength = '中等'
else:
    strength = '弱'

direction = '正' if corr_pearson > 0 else '负'
print(f'\n解读: 资金费率与基差存在 {strength}{direction}相关关系')

In [ ]:
# 8.5 可视化历史关系

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('BTC 价格走势', '资金费率 vs 基差率', '散点图：资金费率 vs 基差率'),
    row_heights=[0.3, 0.4, 0.3],
    vertical_spacing=0.08
)

# 子图1: 价格
fig.add_trace(
    go.Scatter(x=df_analysis.index, y=df_analysis['spot_price'], 
               mode='lines', name='现货价格', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df_analysis.index, y=df_analysis['futures_price'], 
               mode='lines', name='合约价格', line=dict(color='orange', dash='dot')),
    row=1, col=1
)

# 子图2: 资金费率和基差
fig.add_trace(
    go.Scatter(x=df_analysis.index, y=df_analysis['funding_rate_pct'], 
               mode='lines', name='资金费率 (%)', line=dict(color='green')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=df_analysis.index, y=df_analysis['basis_rate'], 
               mode='lines', name='基差率 (%)', line=dict(color='red')),
    row=2, col=1
)
fig.add_hline(y=0, line_dash='dash', line_color='gray', row=2, col=1)

# 子图3: 散点图
fig.add_trace(
    go.Scatter(
        x=df_analysis['basis_rate'], 
        y=df_analysis['funding_rate_pct'],
        mode='markers',
        marker=dict(size=3, color='purple', opacity=0.5),
        name='数据点'
    ),
    row=3, col=1
)

# 添加趋势线
slope, intercept, r_value, p_value, std_err = stats.linregress(
    df_analysis['basis_rate'].dropna(), 
    df_analysis['funding_rate_pct'].dropna()
)
x_line = np.linspace(df_analysis['basis_rate'].min(), df_analysis['basis_rate'].max(), 100)
y_line = slope * x_line + intercept

fig.add_trace(
    go.Scatter(
        x=x_line, y=y_line,
        mode='lines',
        line=dict(color='red', width=2),
        name=f'趋势线 (R²={r_value**2:.3f})'
    ),
    row=3, col=1
)

fig.update_yaxes(title_text='价格 (USDT)', row=1, col=1)
fig.update_yaxes(title_text='百分比 (%)', row=2, col=1)
fig.update_xaxes(title_text='基差率 (%)', row=3, col=1)
fig.update_yaxes(title_text='资金费率 (%)', row=3, col=1)

fig.update_layout(
    title=f'BTC 资金费率与基差关系分析 (相关系数: {corr_pearson:.4f})',
    height=900,
    hovermode='x unified'
)

fig.show()

## 9. 时间序列特征分析

In [ ]:
# 9.1 资金费率的时间特征

df_ts = df_btc_funding.copy()
df_ts['hour'] = df_ts.index.hour
df_ts['day_of_week'] = df_ts.index.dayofweek
df_ts['month'] = df_ts.index.month

# 按结算时间统计
hourly_stats = df_ts.groupby('hour')['funding_rate_pct'].agg(['mean', 'std', 'count'])

print('📊 按结算时间统计 (UTC)')
print('=' * 50)
for hour in [0, 8, 16]:
    if hour in hourly_stats.index:
        stats_row = hourly_stats.loc[hour]
        print(f'{hour:02d}:00 - 平均: {stats_row["mean"]:.4f}%, 标准差: {stats_row["std"]:.4f}%')

In [ ]:
# 9.2 资金费率的分布特征

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 直方图
axes[0, 0].hist(df_ts['funding_rate_pct'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=0, color='red', linestyle='--', label='零线')
axes[0, 0].axvline(x=df_ts['funding_rate_pct'].mean(), color='green', linestyle='-', label=f'均值: {df_ts["funding_rate_pct"].mean():.4f}%')
axes[0, 0].set_xlabel('资金费率 (%)')
axes[0, 0].set_ylabel('频数')
axes[0, 0].set_title('资金费率分布')
axes[0, 0].legend()

# 箱线图按结算时间
data_by_hour = [df_ts[df_ts['hour'] == h]['funding_rate_pct'].values for h in [0, 8, 16]]
bp = axes[0, 1].boxplot(data_by_hour, labels=['00:00 UTC', '08:00 UTC', '16:00 UTC'])
axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0, 1].set_ylabel('资金费率 (%)')
axes[0, 1].set_title('按结算时间分布')

# 箱线图按星期
data_by_day = [df_ts[df_ts['day_of_week'] == d]['funding_rate_pct'].values for d in range(7)]
axes[1, 0].boxplot(data_by_day, labels=['周一', '周二', '周三', '周四', '周五', '周六', '周日'])
axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_ylabel('资金费率 (%)')
axes[1, 0].set_title('按星期分布')

# QQ图
from scipy import stats as scipy_stats
scipy_stats.probplot(df_ts['funding_rate_pct'].dropna(), dist='norm', plot=axes[1, 1])
axes[1, 1].set_title('Q-Q 图（正态性检验）')

plt.tight_layout()
plt.show()

# 正态性检验
statistic, p_value = scipy_stats.shapiro(df_ts['funding_rate_pct'].dropna()[:5000])  # 取前5000个样本
print(f'\n📊 Shapiro-Wilk 正态性检验:')
print(f'   统计量: {statistic:.4f}')
print(f'   p 值: {p_value:.4e}')
print(f'   结论: {"服从" if p_value > 0.05 else "不服从"}正态分布')

In [ ]:
# 9.3 资金费率的自相关性

from pandas.plotting import autocorrelation_plot

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 自相关图
autocorrelation_plot(df_ts['funding_rate_pct'].dropna(), ax=axes[0])
axes[0].set_title('资金费率自相关图')
axes[0].set_xlim([0, 50])

# 滞后相关热力图
lags = list(range(1, 11))
lag_corr = [df_ts['funding_rate_pct'].autocorr(lag=i) for i in lags]

axes[1].bar(lags, lag_corr, color='steelblue', edgecolor='black')
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_xlabel('滞后期数')
axes[1].set_ylabel('自相关系数')
axes[1].set_title('资金费率滞后相关')
axes[1].set_xticks(lags)

plt.tight_layout()
plt.show()

print('\n📊 滞后相关系数:')
for i, corr in zip(lags[:5], lag_corr[:5]):
    print(f'   滞后 {i} 期: {corr:.4f}')

---

# 第四部分：高阶应用

---

## 10. 套利策略设计

### 10.1 资金费率套利原理

**核心思路**：通过在现货和永续合约市场建立对冲头寸，赚取资金费率收益。

```
┌─────────────────────────────────────────────────────────────────┐
│                    正资金费率套利策略                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   条件：资金费率 > 0（多头支付空头）                            │
│                                                                 │
│   操作：                                                        │
│   ┌─────────────┐      ┌─────────────┐                         │
│   │   现货市场   │      │   合约市场   │                         │
│   │   买入 1 BTC │      │   做空 1 BTC │                         │
│   └─────────────┘      └─────────────┘                         │
│          │                    │                                 │
│          └────────────────────┘                                 │
│                    │                                            │
│                    ↓                                            │
│          ┌─────────────────────┐                               │
│          │  价格风险完全对冲    │                               │
│          │  净收益 = 资金费率   │                               │
│          └─────────────────────┘                               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 10.2 成本分析

| 成本类型 | Maker | Taker |
|----------|-------|-------|
| 现货手续费 | 0.10% | 0.10% |
| 合约手续费 | 0.02% | 0.04% |
| 滑点成本 | ~0.02% | ~0.05% |
| **开仓总成本** | ~0.14% | ~0.19% |
| **平仓总成本** | ~0.14% | ~0.19% |

In [ ]:
# 10.3 套利收益计算器

class FundingArbitrageCalculator:
    """
    资金费率套利收益计算器
    """
    
    def __init__(
        self,
        spot_fee: float = 0.001,       # 现货手续费 0.1%
        futures_fee: float = 0.0004,   # 合约手续费 0.04% (taker)
        slippage: float = 0.0003       # 滑点 0.03%
    ):
        self.spot_fee = spot_fee
        self.futures_fee = futures_fee
        self.slippage = slippage
        
        # 计算总成本
        self.open_cost = spot_fee + futures_fee + slippage
        self.close_cost = spot_fee + futures_fee + slippage
        self.total_cost = self.open_cost + self.close_cost
    
    def calculate_breakeven_periods(self, funding_rate: float) -> float:
        """
        计算盈亏平衡需要的期数
        
        参数:
            funding_rate: 单期资金费率（小数形式，如 0.0001 表示 0.01%）
        
        返回:
            达到盈亏平衡需要的期数
        """
        if funding_rate <= 0:
            return float('inf')
        return self.total_cost / funding_rate
    
    def calculate_net_return(
        self, 
        funding_rate: float, 
        periods: int,
        include_costs: bool = True
    ) -> dict:
        """
        计算净收益
        
        参数:
            funding_rate: 单期资金费率
            periods: 持有期数
            include_costs: 是否包含开平仓成本
        
        返回:
            收益详情字典
        """
        gross_return = funding_rate * periods
        costs = self.total_cost if include_costs else 0
        net_return = gross_return - costs
        
        return {
            'gross_return_pct': gross_return * 100,
            'costs_pct': costs * 100,
            'net_return_pct': net_return * 100,
            'periods': periods,
            'days': periods / 3,  # 8小时/期
            'annualized_return_pct': (net_return / (periods / 3) * 365) * 100 if periods > 0 else 0
        }
    
    def is_profitable(
        self, 
        funding_rate: float, 
        min_periods: int = 30
    ) -> tuple:
        """
        判断套利是否有利可图
        
        返回:
            (是否盈利, 收益详情)
        """
        result = self.calculate_net_return(funding_rate, min_periods)
        return result['net_return_pct'] > 0, result
    
    def print_summary(self):
        """打印成本汇总"""
        print('=' * 50)
        print('📊 套利成本汇总')
        print('=' * 50)
        print(f'现货手续费:   {self.spot_fee*100:.3f}%')
        print(f'合约手续费:   {self.futures_fee*100:.3f}%')
        print(f'滑点成本:     {self.slippage*100:.3f}%')
        print(f'-' * 50)
        print(f'开仓总成本:   {self.open_cost*100:.3f}%')
        print(f'平仓总成本:   {self.close_cost*100:.3f}%')
        print(f'**总成本**:   {self.total_cost*100:.3f}%')
        print('=' * 50)

# 创建计算器实例
arb_calc = FundingArbitrageCalculator()
arb_calc.print_summary()

In [ ]:
# 10.4 不同资金费率下的套利收益分析

funding_rates = [0.0001, 0.0003, 0.0005, 0.001, 0.002, 0.005, 0.01]
holding_periods = [3, 9, 30, 90]  # 1天, 3天, 10天, 30天

print('\n' + '=' * 80)
print('📊 不同资金费率和持有期的套利收益分析')
print('=' * 80)
print(f'\n{"资金费率":^12} {"盈亏平衡":^15} {"1天净收益":^12} {"3天净收益":^12} {"10天净收益":^12} {"年化收益":^12}')
print('-' * 80)

for fr in funding_rates:
    breakeven = arb_calc.calculate_breakeven_periods(fr)
    r1 = arb_calc.calculate_net_return(fr, 3)    # 1天 = 3期
    r3 = arb_calc.calculate_net_return(fr, 9)    # 3天 = 9期
    r10 = arb_calc.calculate_net_return(fr, 30)  # 10天 = 30期
    
    # 年化收益（假设持续投资）
    annual = fr * 3 * 365 * 100 - arb_calc.total_cost * 100  # 简化计算
    
    print(f'{fr*100:>8.3f}%    {breakeven:>6.1f}期({breakeven/3:>4.1f}天)   '
          f'{r1["net_return_pct"]:>8.3f}%    {r3["net_return_pct"]:>8.3f}%    '
          f'{r10["net_return_pct"]:>8.3f}%    {annual:>8.2f}%')

print('=' * 80)
print('\n💡 结论：')
print(f'   - 资金费率 > {arb_calc.total_cost/3*100:.3f}%（日均成本）时，长期套利有利可图')
print(f'   - 资金费率 > {arb_calc.total_cost*100:.3f}%（总成本）时，单次套利即可盈利')

In [ ]:
# 10.5 基于历史数据的套利回测

def backtest_funding_arbitrage(
    df_funding: pd.DataFrame,
    initial_capital: float = 10000,
    min_funding_rate: float = 0.0003,  # 最低开仓资金费率
    exit_funding_rate: float = 0.0     # 平仓资金费率阈值
) -> dict:
    """
    回测资金费率套利策略
    
    策略规则：
    - 当资金费率 > min_funding_rate 时开仓
    - 当资金费率 < exit_funding_rate 时平仓
    """
    calc = FundingArbitrageCalculator()
    
    capital = initial_capital
    position = 0  # 0: 无仓位, 1: 有仓位
    
    equity_curve = [initial_capital]
    trades = []
    total_fees = 0
    total_funding_earned = 0
    
    for timestamp, row in df_funding.iterrows():
        funding_rate = row['fundingRate']
        
        # 开仓条件
        if position == 0 and funding_rate >= min_funding_rate:
            # 开仓
            open_fee = capital * calc.open_cost
            capital -= open_fee
            total_fees += open_fee
            position = 1
            trades.append({'time': timestamp, 'action': 'OPEN', 'rate': funding_rate})
        
        # 持仓中收取资金费率
        elif position == 1:
            if funding_rate > 0:
                # 收取正资金费率
                funding_income = capital * funding_rate
                capital += funding_income
                total_funding_earned += funding_income
            else:
                # 支付负资金费率
                funding_cost = capital * abs(funding_rate)
                capital -= funding_cost
            
            # 平仓条件
            if funding_rate < exit_funding_rate:
                close_fee = capital * calc.close_cost
                capital -= close_fee
                total_fees += close_fee
                position = 0
                trades.append({'time': timestamp, 'action': 'CLOSE', 'rate': funding_rate})
        
        equity_curve.append(capital)
    
    # 计算指标
    equity_series = pd.Series(equity_curve)
    rolling_max = equity_series.expanding().max()
    drawdown = (equity_series - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    
    total_return = (capital - initial_capital) / initial_capital
    days = (df_funding.index[-1] - df_funding.index[0]).days
    annual_return = (1 + total_return) ** (365 / max(days, 1)) - 1
    
    return {
        'initial_capital': initial_capital,
        'final_capital': capital,
        'total_return_pct': total_return * 100,
        'annual_return_pct': annual_return * 100,
        'max_drawdown_pct': max_drawdown * 100,
        'total_fees': total_fees,
        'total_funding_earned': total_funding_earned,
        'num_trades': len([t for t in trades if t['action'] == 'OPEN']),
        'equity_curve': equity_curve,
        'trades': trades,
        'days': days
    }

# 执行回测
print('📊 执行套利策略回测...\n')
result = backtest_funding_arbitrage(
    df_btc_funding,
    initial_capital=10000,
    min_funding_rate=0.0003,  # 0.03% 最低开仓费率
    exit_funding_rate=0.0001  # 0.01% 平仓阈值
)

print('=' * 50)
print('📈 回测结果')
print('=' * 50)
print(f'回测周期:     {result["days"]} 天')
print(f'初始资金:     ${result["initial_capital"]:,.2f}')
print(f'最终资金:     ${result["final_capital"]:,.2f}')
print(f'总收益:       {result["total_return_pct"]:.2f}%')
print(f'年化收益:     {result["annual_return_pct"]:.2f}%')
print(f'最大回撤:     {result["max_drawdown_pct"]:.2f}%')
print(f'交易次数:     {result["num_trades"]}')
print(f'总手续费:     ${result["total_fees"]:.2f}')
print(f'资金费率收益: ${result["total_funding_earned"]:.2f}')
print('=' * 50)

In [ ]:
# 10.6 回测资金曲线可视化

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=result['equity_curve'],
    mode='lines',
    name='账户权益',
    line=dict(color='blue', width=2)
))

fig.add_hline(
    y=result['initial_capital'], 
    line_dash='dash', 
    line_color='gray',
    annotation_text='初始资金'
)

fig.update_layout(
    title=f'资金费率套利回测 - 年化收益: {result["annual_return_pct"]:.2f}%, 最大回撤: {result["max_drawdown_pct"]:.2f}%',
    xaxis_title='期数',
    yaxis_title='账户权益 (USDT)',
    height=500
)

fig.show()

## 11. 预测模型构建

In [ ]:
# 11.1 特征工程

def create_prediction_features(df: pd.DataFrame, target_col: str = 'fundingRate', look_back: int = 10) -> pd.DataFrame:
    """
    创建预测资金费率的特征
    
    特征类型：
    1. 滞后特征：过去 N 期的资金费率
    2. 滚动统计：均值、标准差、最大值、最小值
    3. 变化率特征：差分、百分比变化
    4. 时间特征：小时、星期
    """
    df_feat = df.copy()
    
    # 1. 滞后特征
    for i in range(1, look_back + 1):
        df_feat[f'lag_{i}'] = df_feat[target_col].shift(i)
    
    # 2. 滚动统计
    for window in [3, 7, 14, 21]:
        df_feat[f'rolling_mean_{window}'] = df_feat[target_col].shift(1).rolling(window=window).mean()
        df_feat[f'rolling_std_{window}'] = df_feat[target_col].shift(1).rolling(window=window).std()
        df_feat[f'rolling_max_{window}'] = df_feat[target_col].shift(1).rolling(window=window).max()
        df_feat[f'rolling_min_{window}'] = df_feat[target_col].shift(1).rolling(window=window).min()
    
    # 3. 变化率特征
    df_feat['diff_1'] = df_feat[target_col].diff(1)
    df_feat['diff_3'] = df_feat[target_col].diff(3)
    df_feat['pct_change_1'] = df_feat[target_col].pct_change(1)
    df_feat['pct_change_3'] = df_feat[target_col].pct_change(3)
    
    # 4. 时间特征
    df_feat['hour'] = df_feat.index.hour
    df_feat['day_of_week'] = df_feat.index.dayofweek
    
    # 5. 趋势特征
    df_feat['trend_3'] = (df_feat[target_col].shift(1) > df_feat[target_col].shift(4)).astype(int)
    df_feat['trend_7'] = (df_feat[target_col].shift(1) > df_feat[target_col].shift(8)).astype(int)
    
    return df_feat

# 创建特征
df_ml = create_prediction_features(df_btc_funding)
print(f'✅ 特征工程完成')
print(f'   原始列数: {len(df_btc_funding.columns)}')
print(f'   特征列数: {len(df_ml.columns)}')

In [ ]:
# 11.2 准备训练数据

# 定义特征列
feature_cols = [col for col in df_ml.columns if col.startswith(('lag_', 'rolling_', 'diff_', 'pct_', 'hour', 'day_', 'trend_'))]
target_col = 'fundingRate'

print(f'特征数量: {len(feature_cols)}')
print(f'特征列表: {feature_cols[:10]}...')

# 准备数据
X = df_ml[feature_cols].copy()
y = df_ml[target_col].copy()

# 删除缺失值
valid_idx = ~(X.isna().any(axis=1) | y.isna())
X = X[valid_idx]
y = y[valid_idx]

print(f'\n有效样本数: {len(X)}')

# 时间序列分割（80% 训练，20% 测试）
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f'训练集: {len(X_train)} 样本')
print(f'测试集: {len(X_test)} 样本')

In [ ]:
# 11.3 训练多个模型（如果 sklearn 可用）

try:
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LinearRegression, Ridge, Lasso
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    # 标准化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 定义模型
    models = {
        'Linear': LinearRegression(),
        'Ridge': Ridge(alpha=1.0),
        'Lasso': Lasso(alpha=0.0001),
        'RandomForest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
    }
    
    # 训练和评估
    print('\n' + '=' * 70)
    print('📊 模型训练与评估')
    print('=' * 70)
    print(f'{"模型":<20} {"MAE":>12} {"RMSE":>12} {"R²":>12}')
    print('-' * 70)
    
    results = {}
    for name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {'model': model, 'mae': mae, 'rmse': rmse, 'r2': r2, 'y_pred': y_pred}
        print(f'{name:<20} {mae*100:>10.4f}% {rmse*100:>10.4f}% {r2:>10.4f}')
    
    print('=' * 70)
    
    # 找出最佳模型
    best_model_name = min(results, key=lambda x: results[x]['mae'])
    print(f'\n🏆 最佳模型: {best_model_name}')
    
    SKLEARN_AVAILABLE = True
    
except ImportError:
    print('⚠️ sklearn 未安装，跳过机器学习模型训练')
    print('   运行: pip install scikit-learn')
    SKLEARN_AVAILABLE = False

In [ ]:
# 11.4 预测结果可视化

if SKLEARN_AVAILABLE:
    best_result = results[best_model_name]
    
    fig = go.Figure()
    
    # 实际值
    fig.add_trace(go.Scatter(
        x=y_test.index,
        y=y_test.values * 100,
        mode='lines',
        name='实际值',
        line=dict(color='blue', width=2)
    ))
    
    # 预测值
    fig.add_trace(go.Scatter(
        x=y_test.index,
        y=best_result['y_pred'] * 100,
        mode='lines',
        name=f'预测值 ({best_model_name})',
        line=dict(color='red', width=2, dash='dash')
    ))
    
    fig.add_hline(y=0, line_dash='dot', line_color='gray')
    
    fig.update_layout(
        title=f'资金费率预测 - {best_model_name} (MAE={best_result["mae"]*100:.4f}%, R²={best_result["r2"]:.4f})',
        xaxis_title='时间',
        yaxis_title='资金费率 (%)',
        height=500,
        hovermode='x unified'
    )
    
    fig.show()
    
    # 预测下一期
    last_features = X.iloc[[-1]]
    last_features_scaled = scaler.transform(last_features)
    next_prediction = results[best_model_name]['model'].predict(last_features_scaled)[0]
    
    print(f'\n🔮 预测下一期资金费率: {next_prediction*100:.4f}%')
    print(f'📊 预测年化费率: {next_prediction*3*365*100:.2f}%')

## 12. 风险管理与实战建议

### 12.1 主要风险

```
┌─────────────────────────────────────────────────────────────────┐
│                      资金费率套利的主要风险                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. 资金费率反转风险                                           │
│     └── 正费率突然变为负费率，导致亏损                         │
│     └── 应对：设置止损，监控费率变化                           │
│                                                                 │
│  2. 基差波动风险                                               │
│     └── 开仓后基差扩大，影响平仓收益                           │
│     └── 应对：选择基差稳定的时机开仓                           │
│                                                                 │
│  3. 流动性风险                                                 │
│     └── 大额交易可能面临滑点                                   │
│     └── 应对：分批建仓，选择流动性好的交易对                   │
│                                                                 │
│  4. 保证金风险                                                 │
│     └── 合约端可能面临强平风险                                 │
│     └── 应对：使用较低杠杆，保持充足保证金                     │
│                                                                 │
│  5. 交易所风险                                                 │
│     └── 交易所宕机、API 故障等                                 │
│     └── 应对：多交易所分散，保持紧急平仓能力                   │
│                                                                 ���
│  6. 资金成本风险                                               │
│     └── 长期占用资金的机会成本                                 │
│     └── 应对：计算真实年化收益，与其他投资对比                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 12.2 风险管理策略

| 风险类型 | 管理策略 | 具体措施 |
|----------|----------|----------|
| 费率反转 | 动态止损 | 当费率连续 N 期为负时平仓 |
| 基差波动 | 开仓时机选择 | 等待基差收窄时开仓 |
| 流动性 | 仓位控制 | 单笔不超过市场深度的 1% |
| 保证金 | 低杠杆 | 合约端使用 1-3x 杠杆 |
| 交易所 | 分散 | 多交易所同时操作 |

### 12.3 实战建议

1. **入场条件**
   - 资金费率 > 0.03%（年化 > 32%）
   - 基差率 < 0.1%（避免过高溢价）
   - 市场波动率适中

2. **仓位管理**
   - 单币种仓位不超过总资金的 30%
   - 合约端使用 2-3x 杠杆
   - 保留 20% 现金应对极端情况

3. **退出条件**
   - 资金费率连续 3 期 < 0.01%
   - 基差率突然扩大超过 0.2%
   - 达到目标收益率

4. **监控频率**
   - 每 8 小时检查一次资金费率
   - 每小时监控基差变化
   - 设置自动报警机制

In [ ]:
# 12.4 实时套利机会扫描

def scan_arbitrage_opportunities(
    symbols: list,
    min_annual_return: float = 10.0,
    max_basis_rate: float = 0.15
) -> pd.DataFrame:
    """
    扫描套利机会
    
    参数:
        symbols: 交易对列表
        min_annual_return: 最小年化收益要求 (%)
        max_basis_rate: 最大基差率 (%)
    """
    calc = FundingArbitrageCalculator()
    opportunities = []
    
    print(f'\n📊 扫描 {len(symbols)} 个交易对...\n')
    
    for symbol in symbols:
        try:
            # 获取资金费率
            funding = get_current_funding_rate(symbol)
            funding_rate = funding['funding_rate'] / 100  # 转换为小数
            
            if funding_rate <= 0:
                continue
            
            # 获取基差
            basis = get_current_basis(symbol)
            basis_rate = abs(basis['basis_rate'])
            
            # 计算预期收益
            annual_return = calc.calculate_net_return(funding_rate, 1095)['annualized_return_pct']
            breakeven_days = calc.calculate_breakeven_periods(funding_rate) / 3
            
            # 筛选条件
            if annual_return >= min_annual_return and basis_rate <= max_basis_rate:
                opportunities.append({
                    'symbol': symbol.replace('/USDT:USDT', ''),
                    'funding_rate_pct': funding_rate * 100,
                    'annual_return_pct': annual_return,
                    'basis_rate_pct': basis['basis_rate'],
                    'breakeven_days': breakeven_days,
                    'spot_price': basis['spot_price'],
                })
        except Exception as e:
            continue
    
    if not opportunities:
        print('❌ 未发现符合条件的套利机会')
        return pd.DataFrame()
    
    df = pd.DataFrame(opportunities).sort_values('annual_return_pct', ascending=False)
    print(f'✅ 发现 {len(df)} 个套利机会！\n')
    
    return df

# 扫描主流币种
SCAN_SYMBOLS = [
    'BTC/USDT:USDT', 'ETH/USDT:USDT', 'BNB/USDT:USDT', 'SOL/USDT:USDT',
    'XRP/USDT:USDT', 'ADA/USDT:USDT', 'DOGE/USDT:USDT', 'AVAX/USDT:USDT',
    'DOT/USDT:USDT', 'LINK/USDT:USDT', 'UNI/USDT:USDT', 'ATOM/USDT:USDT',
    'LTC/USDT:USDT', 'ETC/USDT:USDT', 'FIL/USDT:USDT', 'NEAR/USDT:USDT'
]

df_opportunities = scan_arbitrage_opportunities(
    SCAN_SYMBOLS,
    min_annual_return=5.0,  # 最低 5% 年化
    max_basis_rate=0.2      # 最大 0.2% 基差
)

if not df_opportunities.empty:
    display(df_opportunities)

---

## 总结

### 核心要点回顾

1. **资金费率**是永续合约特有的机制，用于使合约价格锚定现货价格

2. **基差**反映了市场对未来价格的预期，与资金费率高度相关但不完全相同

3. **资金费率与基差的关系**：
   - 正基差（溢价）→ 正资金费率
   - 负基差（折价）→ 负资金费率
   - 资金费率的调整会反过来影响基差

4. **套利策略**的核心是通过现货-合约对冲赚取资金费率，需要考虑：
   - 开平仓成本（约 0.3%）
   - 资金费率的稳定性
   - 基差波动风险

5. **风险管理**是成功的关键：
   - 设置明确的入场/退出条件
   - 使用低杠杆
   - 分散投资

### 后续学习方向

- [ ] 深度学习时间序列预测（LSTM、Transformer）
- [ ] 多交易所套利策略
- [ ] 高频交易与订单簿分析
- [ ] 期权与永续合约的组合策略